In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from mhqa.dvc import load_experiments

filepaths = list(Path("../../tmp/musique-sweep/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

7 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'validation'}},
  'retrieve': {'model_type': 't5', 'top_k': 3},
  'qa': {'model': 'llama-3.3-70b-tgi',
   'temperature': 0.1,
   'technique': 'multihop-decompose'},
  'run': 1},
 'metrics': {}}

In [3]:
for exp in experiments:
    train_params = exp["params"]["train"]
    if train_params.get("ensemble") is None:
        train_params["ensemble"] = "no"

experiments[0]

{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'validation'}},
  'retrieve': {'model_type': 't5', 'top_k': 3},
  'qa': {'model': 'llama-3.3-70b-tgi',
   'temperature': 0.1,
   'technique': 'multihop-decompose'},
  'run': 1},
 'metrics': {}}

In [12]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df


7 experiments before preprocessing


,id,name,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,...,params.qa.technique,params.run,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1
0,workspace,None,bdsaglam/musique-sweep,answerable,train,noop,no,bdsaglam/musique-sweep,answerable,validation,...,multihop-decompose,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ddbea2ac28966363f333a7b1629839dd7c6f6488,kempt-sash,bdsaglam/musique-sweep,answerable,train,bfsrs-light,no,bdsaglam/musique-sweep,answerable,validation,...,agent-simple,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,104150ea5f68249deca30008dbeda671ff5dd73a,score-mice,bdsaglam/musique-sweep,answerable,train,bfsrs-light,no,bdsaglam/musique-sweep,answerable,validation,...,multihop-decompose,1,0.600000,0.716905,0.620000,0.750604,0.580000,0.737667,0.600000,0.662444
3,fbe64ec97aec6dc3d8c520b3e32f73a14473b4cc,None,bdsaglam/musique-sweep,answerable,train,noop,no,bdsaglam/musique-sweep,answerable,validation,...,multihop-decompose,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c5b35756e60415e3c37c577ebe2dc59ef59a364c,None,bdsaglam/musique-sweep,answerable,train,noop,no,bdsaglam/musique-sweep,answerable,validation,...,multihop-decompose,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,workspace,None,bdsaglam/musique-sweep,answerable,train,bfsrs-medium,no,bdsaglam/musique-sweep,answerable,validation,...,agent-simple,1,0.612329,0.736333,0.639776,0.761202,0.617105,0.764483,0.518519,0.606628
6,8710704865d32d83a64d7c1aff7adcf2e32dc877,terse-play,bdsaglam/musique-sweep,answerable,train,miprov2-light,no,bdsaglam/musique-sweep,answerable,validation,...,cot,1,0.606667,0.723042,0.630000,0.729976,0.630000,0.793833,0.560000,0.645315


In [14]:
mask = df["params.evaluation.dataset.path"] == "bdsaglam/musique-sweep"

df = df.loc[mask].copy()

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")
df

1 experiments after preprocessing


,id,name,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,...,params.qa.technique,params.run,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1
2,104150ea5f68249deca30008dbeda671ff5dd73a,score-mice,bdsaglam/musique-sweep,answerable,train,bfsrs-light,no,bdsaglam/musique-sweep,answerable,validation,...,multihop-decompose,1,0.6,0.716905,0.62,0.750604,0.58,0.737667,0.6,0.662444


In [15]:
def sorted_tuple(x):
    return tuple(sorted(x))

run_counts = (
    df.groupby(param_cols[:-1])[
        "params.run"
    ]
    .aggregate(sorted_tuple)
    .reset_index()
)

In [16]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.train.dataset.path
	bdsaglam/musique-sweep
- params.train.dataset.name
	answerable
- params.train.dataset.split
	train
- params.train.optimizer
	bfsrs-light
- params.train.ensemble
	no
- params.evaluation.dataset.path
	bdsaglam/musique-sweep
- params.evaluation.dataset.name
	answerable
- params.evaluation.dataset.split
	validation
- params.retrieve.model_type
	golden
- params.retrieve.top_k
	3.0
- params.qa.model
	llama-3.3-70b-tgi
- params.qa.temperature
	0.1
- params.qa.technique
	multihop-decompose
- params.run
	(1,)


In [17]:
df.to_json('results.jsonl', orient='records', lines=True)

In [18]:
df[["params.qa.technique", "params.train.optimizer", "metrics.exact_match", "metrics.f1"]]

,params.qa.technique,params.train.optimizer,metrics.exact_match,metrics.f1
2,multihop-decompose,bfsrs-light,0.6,0.716905
